In [1]:
! pip install python-dotenv

! pip install openai

! pip install geopy

! pip install azure-cosmos


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Import the required libraries
import time
import os
import json
import uuid
from dotenv import dotenv_values
from openai import OpenAI, AzureOpenAI


#Cosmos DB imports
from azure.cosmos import CosmosClient

In [98]:
env_name = "variable.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

nosql_uri=config["NOSQL_URI"]
cosmos_key = config['NOSQL_PRIMARY_KEY']
cosmos_database = "Account"
cosmos_container = "Transactions"

openai_endpoint = config['AOAI_ENDPOINT']
openai_key = config['AOAI_KEY']
openai_api_version = config['API_VERSION']
openai_embeddings_deployment = config['AOAI_EMBEDDING_DEPLOYMENT']
openai_embeddings_model = config['AOAI_EMBEDDING_DEPLOYMENT_MODEL']
#openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])

# Create the Azure Cosmos DB for NoSQL client
cosmos_client = CosmosClient(url=nosql_uri, credential=cosmos_key)

azure_openai_embeddings = AzureOpenAI(
    api_version=openai_api_version,
    api_key= openai_key,
    azure_endpoint= openai_endpoint,
)

In [89]:
#create database
DATABASE_NAME = "fraud-nosql-db2"
db= cosmos_client.create_database_if_not_exists(
    id=DATABASE_NAME
)
properties = db.read()
print(json.dumps(properties))

{"id": "fraud-nosql-db2", "_rid": "+39sAA==", "_self": "dbs/+39sAA==/", "_etag": "\"0000f401-0000-4700-0000-66ed8ffd0000\"", "_colls": "colls/", "_users": "users/", "_ts": 1726844925}


In [90]:
vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path":"/locationVector",
            "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":1536
        },
    ]
}

indexing_policy = {
    "includedPaths": [
        {
            "path": "/*"
        }
    ],
    "excludedPaths": [
        {
            "path": "/\"_etag\"/?"
        },
        {
            "path": "/locationVector/*"
        }
    ],
    "vectorIndexes": [
        {"path": "/locationVector",
         "type": "diskANN"
        }
    ]
}

In [91]:
from azure.cosmos import PartitionKey, exceptions

CONTAINER_NAME = "fraud-nosql-cont"
try:    
    container = db.create_container_if_not_exists(
                    id=CONTAINER_NAME,
                    partition_key=PartitionKey(path="/TenantId"),
                    indexing_policy=indexing_policy,
                    vector_embedding_policy=vector_embedding_policy)

    properties = container.read()
    print('Container with properties \'{0}\' created'.format(properties))

except exceptions.CosmosResourceExistsError:
    print('A container with id \'{0}\' already exists'.format(id))

Container with properties '{'id': 'fraud-nosql-cont', 'indexingPolicy': {'indexingMode': 'consistent', 'automatic': True, 'includedPaths': [{'path': '/*'}], 'excludedPaths': [{'path': '/"_etag"/?'}, {'path': '/locationVector/*'}], 'vectorIndexes': [{'path': '/locationVector', 'type': 'diskANN'}]}, 'partitionKey': {'paths': ['/TenantId'], 'kind': 'Hash', 'version': 2}, 'conflictResolutionPolicy': {'mode': 'LastWriterWins', 'conflictResolutionPath': '/_ts', 'conflictResolutionProcedure': ''}, 'geospatialConfig': {'type': 'Geography'}, 'vectorEmbeddingPolicy': {'vectorEmbeddings': [{'path': '/locationVector', 'dataType': 'float32', 'dimensions': 1536, 'distanceFunction': 'cosine'}]}, '_rid': '+39sAPLryro=', '_ts': 1726844930, '_self': 'dbs/+39sAA==/colls/+39sAPLryro=/', '_etag': '"0000f601-0000-4700-0000-66ed90020000"', '_docs': 'docs/', '_sprocs': 'sprocs/', '_triggers': 'triggers/', '_udfs': 'udfs/', '_conflicts': 'conflicts/'}' created


In [92]:
def generate_embeddings(lat_lon):
    lat_lon_str = f"{lat_lon[0]},{lat_lon[1]}"
    
    # Call OpenAI to generate embeddings (assuming text input is required)
    response = azure_openai_embeddings.embeddings.create(input=lat_lon_str, model=openai_embeddings_model)
    embeddings = response.model_dump()
    
    time.sleep(0.5)  # To avoid API rate limits
    
    return embeddings['data'][0]['embedding']

In [93]:
from geopy.geocoders import Nominatim

def get_city_coordinates(city_name):
    try:
        # Create a geolocator object using Nominatim service
        geolocator = Nominatim(user_agent="MyAPP")
        
        # Geocode the city name to get location details
        location = geolocator.geocode(city_name)
        
        if location:
            # Extract the latitude and longitude from the location object
            lat = location.latitude
            lon = location.longitude
            return lat, lon
        else:
            print(f"City '{city_name}' not found.")
            return None, None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None, None



In [94]:
# Load text-sample_w_embeddings.json which has embeddings pre-computed
data_file = open(file="data/data_with_tenants.json", mode="r") 

data = json.load(data_file)
data_file.close()

In [96]:
# Take a peek at one data item
print(json.dumps(data[4], indent=2))

{
  "TransactionID": "T5109",
  "Amount": 108.79,
  "Timestamp": "2024-09-15 14:02:38",
  "Location": "Boston",
  "Merchant": "Lyft",
  "Fraud": false,
  "TenantId": "5"
}


In [99]:
# Generate embeddings for each location and store data in cosmos db container
for item in data:
    transaction_id = item["TransactionID"]
    item['id'] = transaction_id
    location = item["Location"]
    location_coord = get_city_coordinates(location)
    location_embeddings = generate_embeddings(location_coord)
    item['locationVector'] = location_embeddings
    item['@search.action'] = 'upload'
   
    print("Creating embeddings for transaction:", transaction_id, end='\r')
    
    # Insert the item into the container
    container.upsert_item(item)    

In [154]:
import numpy as np

def get_average_location_vector(container, tenant_id):
    # SQL query to get the last 'num_purchases' transactions ordered by timestamp
    sql_query = """
    SELECT c.locationVector
    FROM c
    WHERE c.TenantId = @tenant_id
    ORDER BY c.Timestamp DESC
    """
    
    # Parameters for the query
    parameters = [
        {"name": "@tenant_id", "value": tenant_id}
    ]
    
    # Execute the query to get the location vectors
    results = container.query_items(
        query=sql_query,
        parameters=parameters,
        enable_cross_partition_query=True
    )
    
    # Collect the location vectors
    vectors = []
    for result in results:
        vectors.append(result['locationVector'])
    
    
    # If no vectors are found, return None
    if not vectors:
        return None
    
    # Convert the list of vectors into a numpy array
    vectors_np = np.array(vectors)
    
    # Calculate the element-wise average of the vectors
    avg_vector = np.mean(vectors_np, axis=0)
    
    return avg_vector, len(vectors)


In [187]:
def vector_search( current_location_vector, tenant_id, average_location_vector, amount, num_results=5):

    if isinstance(current_location_vector, np.ndarray):
        current_location_vector = current_location_vector.tolist()
    if isinstance(average_location_vector, np.ndarray):
        average_location_vector = average_location_vector.tolist()

    sql_query = """
 SELECT 
        c.TransactionID,
        c.Amount, 
        c.Timestamp, 
        c.Location, 
        c.Merchant,
        c.TenantId, 
        VectorDistance(c.locationVector, @current_location_vector) AS ProximityOfCurrentToLast,
        VectorDistance(@current_location_vector, @average_location_vector) AS ProximityOfAverageToLast
    FROM c
    WHERE 
        VectorDistance(c.locationVector, @current_location_vector) > 0.1
        AND VectorDistance(@current_location_vector, @average_location_vector) > 0.1
        AND c.TenantId = @tenant_id
    """

    # Parameters for the SQL query
    parameters = [
        {"name": "@num_results", "value": num_results},   
        {"name": "@average_location_vector", "value": average_location_vector},  
        {"name": "@current_location_vector", "value": current_location_vector},  
        {"name": "@amount", "value": amount},  # Transaction amount range filtering
        {"name": "@tenant_id", "value": tenant_id},  # Transaction amount range filtering
    ]

    results = container.query_items(
        query=sql_query,
        parameters=parameters,
        enable_cross_partition_query=True
    )

    return list(results), len(list(results))


In [190]:
tenant_id = "10"
# Assuming you already computed the average location vector and current location vector
average_location_vector, lent = get_average_location_vector(container, tenant_id)

current_location_vector = generate_embeddings(get_city_coordinates("Sweden"))
print(current_location_vector)

# Query parameters
query = "Recent purchase at a grocery store"
amount = 1000.0  # The current transaction amount

# Perform vector search
results, lent = vector_search(current_location_vector,tenant_id, average_location_vector, amount, num_results=5)

for result in results:
    print(result)


11
[0.02148537151515484, 0.003992547746747732, -0.002821290632709861, -0.012764645740389824, 0.014441392384469509, -0.011507085524499416, -0.02848825603723526, 0.05569429323077202, 0.013857818208634853, -0.02819235809147358, -0.030477337539196014, -0.004167208913713694, -0.014605779200792313, -0.018986690789461136, -0.008087837137281895, 0.056318964809179306, -0.022504571825265884, -0.008967307396233082, -0.017819544300436974, 0.006559038534760475, -0.013356437906622887, -0.027140282094478607, -0.03300889581441879, 0.003174722660332918, -0.017753789201378822, 0.012312580831348896, 0.002685671439394355, 0.017309943214058876, -0.032795194536447525, 0.00029255737899802625, -0.00716727040708065, -0.0007844339124858379, 0.011161873117089272, 0.007052199449390173, 0.036658287048339844, 0.012320800684392452, 0.021797707304358482, 0.0004122515965718776, 0.009254984557628632, 0.009748145006597042, 0.01732638292014599, 0.002790468279272318, -0.01918395422399044, 0.05286683887243271, -0.037611730

In [193]:
def perform_search(tenant_id, city, query, amount):

    average_location_vector, lent = get_average_location_vector(container, tenant_id)
    
    # Generate embeddings for the current location
    current_location_vector = generate_embeddings(get_city_coordinates(city))
    
    # Perform vector search
    results, lent = vector_search(current_location_vector, tenant_id, average_location_vector, amount, num_results=5)
    
    # Format results
    result_text = ""
    for result in results:
        result_text += f"TransactionID: {result['TransactionID']}, SimilarityScore: {result['SimilarityScore']}\n"
    
    return results

In [196]:
! pip install gradio

  You can safely remove it manually.

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/149.4 kB ? eta -:--:--
     ---------- -------------------------- 41.0/149.4 kB 487.6 kB/s eta 0:00:01
     ---------------------- -------------- 92.2/149.4 kB 655.4 kB/s eta 0:00:01
     ------------------------------------ 149.4/149.4 kB 892.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/434.9 kB ? eta -:--:--
   ------ --------------------------------- 71.7/434.9 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 434.9/434.9 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------- ----------------- 1.1/1.9 MB 35.0 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.3
    Uninstalling pydantic_core-2.23.3:
      Successfully uninstall

In [198]:
! pip install pydantic==1.10.7


     ---------------------------------------- 0.0/146.6 kB ? eta -:--:--
     ----- ------------------------------- 20.5/146.6 kB 640.0 kB/s eta 0:00:01
     ------------------ ------------------ 71.7/146.6 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 146.6/146.6 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.1 MB 16.8 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.14.0 requires pydantic>=2.0, but you have pydantic 1.10.7 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [199]:
import gradio as gr


with gr.Blocks() as demo:
    gr.Markdown("## Fraud Detection Vector Search")

    tenant_id = gr.Number(label="Tenant ID", value=10)
    city = gr.Textbox(label="City", placeholder="Enter the city, e.g., Sweden")
    query = gr.Textbox(label="Query", placeholder="e.g., Recent purchase at a grocery store")
    amount = gr.Number(label="Transaction Amount", value=100.0)

    search_button = gr.Button("Perform Vector Search")
    output = gr.Textbox(label="Results")

    search_button.click(perform_search, inputs=[tenant_id, city, query, amount], outputs=output)

# Launch the Gradio app
demo.launch()

ImportError: cannot import name 'RootModel' from 'pydantic' (C:\Users\Khelan Modi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydantic\__init__.cp311-win_amd64.pyd)